In [1]:
import sqlite3
import pandas as pd
import datetime
from contextlib import closing
pd.options.display.max_columns = 1000

In [2]:
class SqliteCore(object):
    def __init__(self, db_path):
        self.connection = sqlite3.connect(db_path)
        self.primary_idx_dict = {}
        
    def creat_table(self, table_name, para_list):
        sql_cmd = 'CREATE TABLE IF NOT EXISTS {}('.format(table_name)
        for idx in range(len(para_list) - 1):
            para_list[idx] += ','
        for para in para_list:
            sql_cmd += para
        sql_cmd += ')'
        self.connection.execute(sql_cmd)
    
    def _add_data(self, table_name, data_df, exists_type='append'):
        data_df.to_sql(table_name, con=self.connection, if_exists=exists_type, index=False)
        
    def _get_primary_idx(self, table_name, primary_key=None):
        if primary_key is None:
            primary_key = table_name + '_id'
        res = pd.read_sql('select * from {} order by {} desc'.format(table_name, primary_key), sql_core.connection)
        self.primary_idx_dict[table_name] = res.loc[0][primary_key]
        return res.loc[0][primary_key]

In [3]:
sql_core = SqliteCore('test.db')
sql_core.creat_table('instrument', [
    'instrument_id int primary key not null',
    'description text not null',
    'ticker text not null',
    'maturity datetime',
    'strike real',
    'option_type text'
])

sql_core.creat_table('market', [
    'market_id int primary key not null',
    'instrument_id',
    'timestamp datetime not null',
    'bid real',
    'ask real',
    'mid real',
    'delta real',
    'IV real',
    'gamma real',
    'thela real',
    'vega real',
    'foreign key (instrument_id) references instrument (instrument_id)'
])

sql_core.creat_table('trade', [
    'trade_id int primary key not null',
    'instrument_id',
    'timestamp datetime not null',
    'qty real not null',
    'price real not null',
    'reason text',
    'foreign key (instrument_id) references instrument (instrument_id)'
])

sql_core.creat_table('position', [
    'position_id int primary key not null',
    'instrument_id',
    'replace_position_id',
    'startdate datetime not null',
    'enddate datetime not null',
    'foreign key (instrument_id) references instrument (instrument_id)',
    'foreign key (replace_position_id) references position (replace_position_id)'
])

# order是关键字
sql_core.creat_table('my_order', [
    'order_id int primary key not null',
    'instrument_id',
    'timestamp datetime not null',
    'qty real not null',
    'order_type real not null',
    'order_price real not null',
    'foreign key (instrument_id) references instrument (instrument_id)'
])

In [4]:
sql_core._get_primary_idx('instrument')
print(sql_core.primary_idx_dict)

{'instrument': 2}


In [5]:
a = pd.read_csv('c3b5b352d23a34cfd555965199c8791c.csv')
a = a.head(2)
print(a)

           symbol           T     K putCall  \
0  AAPL_112522C70  2022-11-25  70.0    CALL   
1  AAPL_120222C70  2022-12-02  70.0    CALL   

                         description exchangeName   bid   ask  last   mark  \
0  AAPL Nov 25 2022 70 Call (Weekly)          OPR  77.8  78.0  80.3  77.90   
1   AAPL Dec 2 2022 70 Call (Weekly)          OPR  77.8  78.1   0.0  77.95   

   bidSize  askSize bidAskSize  lastSize  highPrice  lowPrice  openPrice  \
0       11        8       11X8         0        0.0       0.0        0.0   
1       27        6       27X6         0        0.0       0.0        0.0   

   closePrice  totalVolume  tradeDate            tradeTime  \
0       81.08            0        NaN  2022-11-23 13:41:05   
1       81.12            0        NaN  1969-12-31 19:00:00   

             quoteTime  netChange  volatility  delta  gamma  theta  vega  \
0  2022-11-25 11:26:21      -0.78     548.517  0.999    0.0 -0.046   0.0   
1  2022-11-25 11:26:05       0.00     163.080  1.000   

In [16]:
def market_data_entry_api(sql_core, csv_path):
    res_df = pd.DataFrame(columns=[
        'market_id', 'instrument_id', 'timestamp', 'bid',
        'ask', 'mid', 'delta', 'IV', 'gamma', 'thela',
        'vega'
    ])
    cur_market_id = sql_core._get_primary_idx('market') + 1
    df = pd.read_csv(csv_path)
    # 测试中
    df = df.head(2)
    for idx, row in df.iterrows():
        res_df.loc[len(res_df)] = {
            'market_id': cur_market_id,
            'instrument_id': 1,
            'timestamp': datetime.datetime(2022, 1, 22, 16 ,0),
            'bid': row['bid'],
            'ask': row['ask'],
            'mid': (row['bid'] + row['ask']) / 2,
            'delta': row['delta'],
            'IV': row['volatility'],
            'gamma': row['gamma'],
            'thela': row['theta'],
            'vega': row['vega']
        }
        cur_market_id += 1
    sql_core._add_data('market', res_df)
market_data_entry_api(sql_core, 'c3b5b352d23a34cfd555965199c8791c.csv')

In [ ]:
instrument_data = pd.DataFrame({
    'instrument_id': 2,
    'description': 'AAPL_stock',
    'ticker': 'AAPL',
}, index=[0])
sql_core._add_data('instrument', instrument_data)

In [ ]:
pd.read_sql('select * from instrument', sql_core.connection)

In [7]:
abc = pd.DataFrame({
    'market_id': 1,
    'instrument_id': 1,
    'timestamp': datetime.datetime(2022, 1, 22, 16 ,0),
    'bid': 11,
    'ask': 11,
    'mid': 11,
    'delta': 11,
    'IV': 11,
    'gamma': 11,
    'thela': 11,
    'vega': 11
}, index=[0])
sql_core._add_data('market', abc)

In [17]:
pd.read_sql('select * from market', sql_core.connection)

,market_id,instrument_id,timestamp,bid,ask,mid,delta,IV,gamma,thela,vega
0,1,1,2022-01-22 16:00:00,11.0,11.0,11.00,11.000,11.000,11.0,11.000,11.0
1,2,1,2022-01-22 16:00:00,77.8,78.0,77.90,0.999,548.517,0.0,-0.046,0.0
2,3,1,2022-01-22 16:00:00,77.8,78.1,77.95,1.000,163.080,0.0,-0.020,0.0
